In [ ]:
from sasmodels.core import load_model
from sasmodels.bumps_model import Model, Experiment

from sasmodels.data import load_data

from bumps.names import *
from bumps.fitters import fit
from bumps.formatnum import format_uncertainty

**Import data as file**

In [ ]:
import pylab

test_data_load = load_data('cyl_400_20.txt')

#We set some errors for demonstration
test_data_load.dy = 0.2*test_data_load.y

test_data_load

**Prepare scipp Dataset test structure (1D)**

In [ ]:
import scipp as sc
import numpy as np
var_q = sc.Variable(
        dims=['Q'],
        unit=sc.units.one/sc.units.angstrom,
        values=test_data_load.x)
var_iq = sc.Variable(
        dims=['Q'],
        unit=sc.units.one/sc.units.m,
        values=test_data_load.y)

var_dy = sc.Variable(
        dims=['Q'],
        unit=sc.units.one/sc.units.m,
        values=test_data_load.dy)

test_sc_dataset = sc.Dataset(
      {'I(Q)': var_iq, 'dy': var_dy},
      coords={'Q': var_q}
)

test_sc_dataset

**Import data as SCIPP data array**

In [ ]:
from sas.sascalc.dataloader.scipp import convert_dataset
test_data = convert_dataset(test_sc_dataset)

**Set up the model**

In [ ]:
kernel = load_model('cylinder')
pars = dict(radius=35, 
            length=350,
            background=0.0,
            scale=1.0,
            sld=4.0,
            sld_solvent=1.0)
model = Model(kernel, **pars)

# SET THE FITTING PARAMETERS
model.radius.range(1, 50)
model.length.range(1, 500)


**Set up the fit problem with test data and our model**

In [ ]:
M = Experiment(data=test_data, model=model)
problem = FitProblem(M)
print("Initial chisq", problem.chisq_str())
problem.plot()
pylab.show()

**Run the fitting with BUMPS**

In [ ]:
result = fit(problem, method='amoeba')
print("Final chisq", problem.chisq_str())

**Show the fit results**

In [ ]:
for k, v, dv in zip(problem.labels(), result.x, result.dx):
    print(k, ":", format_uncertainty(v, dv))
problem.plot()
pylab.show()